In [3]:
from astroquery.gaia import Gaia
from astropy.table import Table
from matplotlib.pyplot import subplots
import matplotlib.pyplot as plt
plt.style.use('bmh')
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

### Reading in Kepler/Gaia crossmatch data 

In [5]:
kepler_dr3_good = Table.read('Data/kepler_dr3_good.fits', format='fits') # reading in crossmatch data
kepler_dr3_good_df = kepler_dr3_good.to_pandas() # creating a dataframe for later use
kepler_dr3_good_sourceids = kepler_dr3_good[['source_id', 'mh_gspphot']] # to be later uploaded to Gaia database

# since the original crossmatch used the gaia_source_lite table, there are some columns missing which I need
# for the calibration
# checking which parameters that we need (for callibration) are already in the table, and which are not.
parameters_for_callibration = [
    'teff_gspphot', 'logg_gspphot', 'mh_gspphot', 'azero_gspphot', 'ebpminrp_gspphot', 'ag_gspphot', 
    'mg_gspphot', 'ra', 'dec', 'libname_gspphot'
]
required_parameters = []
unneeded_parameters = []

for i in parameters_for_callibration:
    if i in kepler_dr3_good.columns:
        unneeded_parameters += [i]
    else:
        required_parameters += [i]
        
print(f"The parameters we need that AREN'T currently in the dataframe are {', '.join(required_parameters)}")
print(f"The parameters we need that ARE already in the dataframe are {', '.join(unneeded_parameters)}")

The parameters we need that AREN'T currently in the dataframe are azero_gspphot, mg_gspphot, libname_gspphot
The parameters we need that ARE already in the dataframe are teff_gspphot, logg_gspphot, mh_gspphot, ebpminrp_gspphot, ag_gspphot, ra, dec


### Getting the parameters I need from the Gaia query system

In [6]:
# logging in to Gaia

username = 'cromines'
password = "6CPd@3Wp#3HGSk"
Gaia.login(user = username, password = password)

INFO: Login to gaia TAP server [astroquery.gaia.core]
INFO: OK [astroquery.utils.tap.core]
INFO: Login to gaia data server [astroquery.gaia.core]
INFO: OK [astroquery.utils.tap.core]


In [5]:
# uploading source ids to gaia (has already been uploaded)

# Gaia.upload_table(upload_resource = kepler_dr3_good_sourceids, table_name = "kepler_gaia_xmatch")

In [7]:
# Checking columns of my table

full_qualified_table_name = 'user_' + username + '.kepler_gaia_xmatch' # table must be referenced in this form
my_table = Gaia.load_table(full_qualified_table_name)

for column in my_table.columns:
    print(column.name)

kepler_gaia_xmatch_oid
source_id
mh_gspphot


In [8]:
# Building the query and running the job (~ 45 seconds)

gaia_source_params = "user.source_id, gaia.azero_gspphot, gaia.libname_gspphot, params.mg_gspphot"

adql_query = f"""
SELECT {gaia_source_params} 
FROM {full_qualified_table_name} AS user 
JOIN gaiadr3.gaia_source AS gaia 
ON gaia.source_id = user.source_id 
JOIN gaiadr3.astrophysical_parameters AS params 
ON params.source_id = user.source_id
"""    

job = Gaia.launch_job_async(adql_query)

# Convert the result to a Pandas DataFrame
results = job.get_results()
df_results = results.to_pandas()
df_results.head(3)

INFO: Query finished. [astroquery.utils.tap.core]


,source_id,azero_gspphot,libname_gspphot,mg_gspphot
0,2050233803028318080,NaN,,NaN
1,2050233601176543104,0.5364,MARCS,5.0881
2,2050230543159814656,0.0064,MARCS,0.3053


### Running the calibration and adding the calibrated metallicity back as a column

In [18]:
# getting all the required parameters for the calibration into one place
calibration_df = pd.merge(df_results, kepler_dr3_good.to_pandas(), on='source_id', how='inner')
calibration_df = calibration_df[[
    'source_id', 'teff_gspphot', 'logg_gspphot', 'mh_gspphot', 'azero_gspphot', 'ebpminrp_gspphot', 
    'ag_gspphot', 'mg_gspphot', 'libname_gspphot', 'ra', 'dec'
]]

# cleaning up the dataframe for the calibrations
clean_calibration_df = calibration_df.dropna(axis=0) # drop rows with NaN in them
clean_calibration_df = clean_calibration_df[clean_calibration_df['libname_gspphot'].isin(['MARCS', 'PHOENIX'])] # only MARCS and PHOENIX
clean_calibration_df = clean_calibration_df.reset_index(drop=True) # reset indexes
# libraries have enough training data to be calibrated
clean_calibration_df.head(3)

,source_id,teff_gspphot,logg_gspphot,mh_gspphot,azero_gspphot,ebpminrp_gspphot,ag_gspphot,mg_gspphot,libname_gspphot,ra,dec
0,2050233601176543104,5616.329590,4.4508,-0.3394,0.5364,0.2373,0.4400,5.0881,MARCS,291.043058,36.593767
1,2050230543159814656,4451.399902,2.1047,-0.3895,0.0064,0.0026,0.0049,0.3053,MARCS,291.055891,36.559885
2,2050231848829944320,5004.754395,4.4394,-0.1710,0.1634,0.0684,0.1295,5.5022,PHOENIX,291.137622,36.577344


In [19]:
from gdr3apcal import GaiaDR3_GSPPhot_cal # from github metallicity calibration tool
calib = GaiaDR3_GSPPhot_cal()
metal_calib = calib.calibrateMetallicity(clean_calibration_df) # calibrating metallicity

Automatically adding cos(b) from given ra and dec assuming degrees.
Automatically adding cos(b) from given ra and dec assuming degrees.


In [20]:
# adding the calibrated metallicities to the dataframe

clean_calibration_df['calibrated_FeH'] = metal_calib
clean_calibration_df = clean_calibration_df[['source_id', 'calibrated_FeH']]
clean_calibration_df.head(3)

,source_id,calibrated_FeH
0,2050233601176543104,-0.041918
1,2050230543159814656,-0.190046
2,2050231848829944320,0.428135


In [21]:
calibrated_kepler_dr3_good = pd.merge(clean_calibration_df, kepler_dr3_good.to_pandas(), on='source_id')
calibrated_kepler_dr3_good.to_csv('Data/calibrated_kepler_dr3_good.csv', index=False) # saving the dataframe as a file
calibrated_kepler_dr3_good.head(3)

,source_id,calibrated_FeH,kepid,ra_kic,dec_kic,random_index,ra,dec,parallax,parallax_error,...,mass_err2,prov_sec,nconfp,nkoi,ntce,jmag,hmag,kmag,planet?,kepler_gaia_mag_diff
0,2050233601176543104,-0.041918,757099,291.04306,36.593811,187127310,291.043058,36.593767,2.798592,0.031155,...,-0.233,b'DSEP',0,0,1,11.656,11.193,11.094,b'none',-0.072974
1,2050230543159814656,-0.190046,757137,291.05588,36.559952,391020929,291.055891,36.559885,1.754854,0.010898,...,-0.947,b'DSEP',0,0,0,7.467,6.853,6.722,b'none',-0.063634
2,2050231848829944320,0.428135,757450,291.13760,36.577381,189196887,291.137622,36.577344,1.231338,0.020312,...,-0.040,b'DSEP',1,1,1,13.665,13.262,13.118,b'conf',-0.104204
